In [133]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import operator

0it [00:00, ?it/s]


In [2]:
with open("train.csv", "r") as f:
    train_text = f.read()
    
rows = train_text.split("\n")
rows_list = []

for i in rows:
    
    data = i.split("|")
    
    if len(data) == 3:
        user_id = data[0]
        movie_name = data[1]
        watch_time = float(data[2])
    
        rows_list.append([user_id, movie_name, watch_time])
        
train = pd.DataFrame(rows_list)
train.columns = ["user_id", "title", "watch_time"]

In [3]:
test = pd.read_csv("test.csv")

In [4]:
with open("movies_meta.csv", "r") as f:
    movies_meta = f.read()

In [5]:
movies = movies_meta.split("\n")[1:]

In [6]:
movies_df = pd.DataFrame()

movies_list = []

for movie in movies:
    
    data = movie.split("|")

    if len(data) == 4:

        movie_name = data[0]
        genre = data[1]

        langs = data[2].split(",")

        movie_length = float(data[3])

        for lang in langs:
            movies_list.append([movie_name, genre, lang, movie_length])

movies_df = pd.DataFrame(movies_list)
movies_df.columns = ["title", "movie_genre", "movie_language", "movie_length"]    

In [7]:
train_with_length = pd.merge(train, movies_df[["title", "movie_length"]], on="title")

In [8]:
train_with_length["ratio_watched"] = train_with_length["watch_time"]/train_with_length["movie_length"]

In [9]:
train_with_length = train_with_length.drop_duplicates()

In [101]:
train_matrix = train_with_length[["user_id", "title", "ratio_watched"]].pivot(index='user_id', columns='title')['ratio_watched']

In [61]:
def num_movies_watched(row):
    return len(row) - row.isnull().sum()

In [64]:
num_movies_watched_df = train_matrix.progress_apply(num_movies_watched, axis = 1)

876597it [03:31, 4135.47it/s]


In [70]:
train_matrix["num_movies"] = num_movies_watched_df

In [74]:
sparse_train_matrix = train_matrix[train_matrix.num_movies > 5]
sparse_train_matrix.drop("num_movies", axis = 1, inplace=True)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
train_matrix.to_pickle("train_matrix.pkl")

SystemError: error return without exception set

In [ ]:
item_item_sim = {}

col_names = sparse_train_matrix.columns.tolist()

res = {}

def get_similar_items(row):
    
    similar_movies = {}
    
    watched_movies = pd.DataFrame(train_matrix.iloc[train_matrix.index.tolist().index(str(row.user_id)), :].dropna()).index.tolist()
    print  row.user_id, watched_movies
    
    for watched_movie in watched_movies:
        
        if watched_movie not in item_item_sim:
            
            item_item_sim[watched_movie] = {}
            
            for i in range(0, len(col_names)):                
                    
                item_item_sim[watched_movie][col_names[i]] = get_cosine(watched_movie, col_names[i])
                    
        for j in item_item_sim[watched_movie]:
            
            if j not in similar_movies:
                similar_movies[j] = item_item_sim[watched_movie][j]
            else:
                similar_movies[j] += item_item_sim[watched_movie][j]
    
    
    for watched_movie in watched_movies:
        del similar_movies[watched_movie]
        
    top_similar_movies = sorted(similar_movies.items(), key=operator.itemgetter(1), reverse=True)[:20]
    
    res[str(row.user_id)] = [i[0] for i in top_similar_movies]
    
test.progress_apply(get_similar_items, axis = 1)

1it [14:41, 881.97s/it]

100146 ['ms dhoni: the untold story', 'nishabd']


2it [15:15, 627.50s/it]

100166 ['ms dhoni: the untold story']
100245 ['bedroom partner', 'julie']


4it [15:46, 443.94s/it]

...

100980 ['kabali']


14it [20:22, 38.33s/it]

10112 ['attharintiki daaredhi', 'jolly llb', 'ms dhoni: the untold story', 'neerja']


15it [21:16, 42.82s/it]

101241 ['2 countries', 'aamayum muyalum', 'action hero biju', 'adi kapyare kootamani', 'christian brothers', 'jamna pyari', 'jawan of vellimala', 'kammattipadam', 'loham', 'lollipop', 'mathai kuzhappakkaranalla', 'oru indian pranaya kadha', 'salalah mobiles', 'teja bhai & family']


In [85]:
def get_cosine(i, j):
    
#     sub_df = pd.concat([sparse_train_matrix.iloc[:, i], sparse_train_matrix.iloc[:, j]], axis = 1)

    return len(sparse_train_matrix[[i, j]].dropna(axis=0, how='any'))

#     sub_df = sub_df.dropna(axis = 0, how = "all")
#     sub_df = sub_df.fillna(0)
#     return 1 - cosine(sub_df.iloc[:, 0], sub_df.iloc[:, 1])

In [32]:
get_cosine(train_matrix.columns.tolist()[0], train_matrix.columns.tolist()[1])

0

In [86]:
cosine_vals = []
col_names = sparse_train_matrix.columns.tolist()

for i in range(0, len(train_matrix.columns.tolist())):
    if i%100 == 0:
        print i
    val = get_cosine(col_names[0], col_names[i])
#     val = get_cosine(0, i)

    cosine_vals.append(val)

0
100
200
300
400
500


KeyboardInterrupt: 

In [113]:
np.unique(train.user_id).tolist()

['0',
 '1',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '100001',
 '100002',
 '100003',
 '100004',
 '100005',
 '100006',
 '100007',
 '100008',
 '100009',
 '10001',
 '100010',
 '100011',
 '100012',
 '100013',
 '100014',
 '100015',
 '100016',
 '100017',
 '100018',
 '100019',
 '10002',
 '100020',
 '100021',
 '100022',
 '100023',
 '100024',
 '100025',
 '100026',
 '100027',
 '100028',
 '100029',
 '10003',
 '100030',
 '100031',
 '100032',
 '100033',
 '100034',
 '100035',
 '100036',
 '100037',
 '100038',
 '100039',
 '10004',
 '100040',
 '100041',
 '100042',
 '100043',
 '100044',
 '100045',
 '100046',
 '100047',
 '100048',
 '100049',
 '10005',
 '100050',
 '100051',
 '100052',
 '100053',
 '100054',
 '100055',
 '100056',
 '100057',
 '100058',
 '100059',
 '10006',
 '100060',
 '100061',
 '100062',
 '100063',
 '100064',
 '100065',
 '100066',
 '100067',
 '100068',
 '100069',
 '10007',
 '100070',
 '100071',
 '100072',
 '100073',
 '100074',
 '100075',
 '100076',
 '100077',
 '100078',
 '100079',
 '10

In [144]:
len(test)

6285